### AI-hub에서 다운받은 데이터를 필요한 클래스만 남기는 코드

In [ ]:
import os
import json

# 입력 및 출력 폴더 경로 설정
input_folder = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/json"    # JSON 파일들이 저장된 폴더 경로
output_folder = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/1_converted"  # 변환된 JSON 파일들을 저장할 폴더 경로

# 남길 클래스 목록 (필요에 따라 이 리스트에 원하는 class_ID를 추가하면 됩니다)
classes_to_keep = ["WO-01", "SO-24", "SO-40", "WO-23", 
                   "SO-28", "SO-20", "WO-03"]

# 출력 폴더가 없으면 생성
os.makedirs(output_folder, exist_ok=True)

# 입력 폴더 내의 모든 JSON 파일 처리
for filename in os.listdir(input_folder):
    if filename.endswith(".json"):
        input_path = os.path.join(input_folder, filename)
        
        # JSON 파일 읽기
        with open(input_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # "Learning_Data_Info." 내의 "Annotations" 리스트 필터링:
        # 남길 클래스 목록에 포함된 항목만 남김
        annotations = data.get("Learning_Data_Info.", {}).get("Annotations", [])
        filtered_annotations = [ann for ann in annotations if ann.get("class_ID") in classes_to_keep]
        
        if "Learning_Data_Info." in data:
            data["Learning_Data_Info."]["Annotations"] = filtered_annotations
        
        # 결과를 출력 폴더에 저장
        output_path = os.path.join(output_folder, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        
        print(f"{filename} 변환 완료")

print("모든 파일 변환이 완료되었습니다.")

### json파일의 경로를 실제 이미지 위치로 변경하는 코드

In [ ]:
import os
import json
import base64

# 변환할 인터넷 JSON 파일들이 있는 폴더와 변환된 파일을 저장할 폴더, 그리고 이미지 파일들이 보관된 폴더를 지정합니다.
input_dir = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/1_converted"      # 인터넷 JSON 파일 폴더 경로로 변경
output_dir = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/2_converted"      # 변환된 LabelMe JSON 파일을 저장할 폴더 경로로 변경
image_dir = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/image"               # 실제 이미지 파일이 보관된 폴더 경로로 변경

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def parse_resolution(resolution_str):
    # 해상도 문자열 예: "1920, 1080" 또는 "1920x1080"
    if "," in resolution_str:
        parts = resolution_str.split(",")
    elif "x" in resolution_str.lower():
        parts = resolution_str.lower().split("x")
    else:
        return None, None
    try:
        width = int(parts[0].strip())
        height = int(parts[1].strip())
        return width, height
    except:
        return None, None

def load_image_as_base64(image_path):
    if os.path.exists(image_path):
        with open(image_path, "rb") as f:
            img_data = f.read()
        return base64.b64encode(img_data).decode("utf-8")
    else:
        return ""

def convert_internet_json_to_labelme(in_data):
    # LabelMe 기본 구조 생성
    labelme_data = {
        "version": "4.5.6",
        "flags": {},
        "shapes": [],
        "imagePath": "",
        "imageData": "",
        "imageHeight": None,
        "imageWidth": None
    }
    
    # 이미지 정보: Source_Data_Info.와 Raw_Data_Info. 사용
    source_info = in_data.get("Source_Data_Info.", {})
    raw_info = in_data.get("Raw_Data_Info.", {})
    src_data = source_info.get("Source_Data", "unknown")
    file_ext = source_info.get("File_extension", "jpg")
    image_filename = f"{src_data}.{file_ext}"
    labelme_data["imagePath"] = image_filename

    # image_dir에 실제 이미지 파일이 존재한다면 base64 인코딩한 값을 채워넣습니다.
    image_path = os.path.join(image_dir, image_filename)
    labelme_data["imageData"] = load_image_as_base64(image_path)
    
    # 해상도 파싱
    resolution = raw_info.get("Resolution", "")
    width, height = parse_resolution(resolution)
    labelme_data["imageWidth"] = width
    labelme_data["imageHeight"] = height
    
    # Learning_Data_Info.에서 어노테이션 정보 처리
    annotations = in_data.get("Learning_Data_Info.", {}).get("Annotations", [])
    for ann in annotations:
        shape = {}
        shape["label"] = ann.get("class_ID", "")
        ann_type = ann.get("type", "").lower()
        
        if ann_type == "polygon":
            points_flat = ann.get("value", [])
            if len(points_flat) % 2 != 0:
                continue
            points = [[points_flat[i], points_flat[i+1]] for i in range(0, len(points_flat), 2)]
            shape["points"] = points
            shape["shape_type"] = "polygon"
        elif ann_type == "bbox":
            bbox = ann.get("value", [])
            if len(bbox) == 4:
                x_min, y_min, x_max, y_max = bbox
                points = [
                    [x_min, y_min],
                    [x_max, y_min],
                    [x_max, y_max],
                    [x_min, y_max]
                ]
                shape["points"] = points
                shape["shape_type"] = "polygon"
            else:
                continue
        else:
            continue
        
        shape["group_id"] = None
        shape["flags"] = {}
        labelme_data["shapes"].append(shape)
    
    return labelme_data

# 폴더 내의 모든 JSON 파일을 변환
for filename in os.listdir(input_dir):
    if filename.endswith(".json"):
        input_path = os.path.join(input_dir, filename)
        with open(input_path, "r", encoding="utf-8") as f:
            try:
                data = json.load(f)
            except Exception as e:
                print(f"Error loading {filename}: {e}")
                continue
            
        labelme_json = convert_internet_json_to_labelme(data)
        
        output_path = os.path.join(output_dir, filename)
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(labelme_json, f, indent=4, ensure_ascii=False)
        
        print(f"Converted {filename} -> {output_path}")


### 전체 데이터에서 클래스별로 몇개가 있는지 확인하는 코드

In [5]:
import os
import json

# JSON 파일들이 저장된 폴더 경로 (실제 경로로 수정)
folder_path = "/home/ted/dev_ws/deeplearing_pj/mask/tolabel/last_f/2_converted"

# 집계할 클래스 목록
classes_to_count = ["WO-01", "SO-24", "SO-40", "WO-23", 
                   "SO-28", "SO-20", "WO-03"]

# 각 클래스별 카운트를 저장할 딕셔너리 초기화
class_counts = {cls: 0 for cls in classes_to_count}

# 폴더 내의 모든 JSON 파일에 대해 처리
for filename in os.listdir(folder_path):
    if filename.endswith(".json"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # JSON 파일 형식에 따라 "shapes" 리스트를 확인 (예제에서는 LabelMe 형식 사용)
        shapes = data.get("shapes", [])
        for shape in shapes:
            label = shape.get("label")
            if label in class_counts:
                class_counts[label] += 1

# 결과 출력
for cls, count in class_counts.items():
    print(f"{cls}: {count}")


WO-01: 3199
SO-24: 1600
SO-40: 3181
WO-23: 1601
SO-28: 1116
SO-20: 1600
WO-03: 1599
